# 论文阅读《Learning visual similarity for product design with convolutional neural networks》
参考：
1. [论文 | 图像检索经典论文解读《Learning visual similarity for product design with convolutional neural networks》](https://www.jianshu.com/p/fd18832dc893?from=groupmessage)
2. [Learning visual similarity for product design with convolutional neural networks](https://cs.colby.edu/courses/S16/cs365/papers/bell-learningProducts-SIG15.pdf)

# 1. idea
1. 该文章研究的是用卷积神经网络学习产品相似性的两个问题，一是对于给定区域的图像，找出与这个区域中包含物品最相似的其它物品，二是对于一个物品，找到包含相似物品的室内设计图。
2. Query1可以解决的问题是人们常常会在装修网站询问“这张图片中的台灯挺好的，从哪里可以买到相似的？”；而Query2可以解决的问题是“这条椅子可以摆放房间的哪里？
3. 在给定物品区域框的情况下，两个问题的本质都在于学习两个物品的相似度，然后利用相似度来完成检索任务。

# 2. 贡献
1. 生成物品与对应场景的数据集；
2. 使用孪生CNN网络(siamese network)并结合contrastive loss & classfication loss完成了整个模型的训练；
3. 将网络应用到实际中；

# 3. 背景介绍
## 3.1 background: 用孪生网络进行距离度量
### 3.1.1 contrastive loss
1. 首先介绍一下正样本和负样本的概念：
  + 正样本：物品q和p是相似的，通常被标注为y=1，那么我们称这样的训练数据对(p,q)是positive pair;
  + 负样本：物品q和n是不相似的，通常被标注为y=0，那么我们称这样的训练数据对(p,n)是negative pair;
2. 如图，我们定义一个边界值Margin(m)来判定两个物品是否相似，例如对于图中的p与q被标注为正样本，在图中体现在两者的特征值会比较相近(圆内); 反之，p与q为负样本，那么他们的特征值之差一定大于m。
![](./img/contrastive_loss.webp)   
接下来我们来学习一下如何用正负样本计算损失值contrastive loss。如下图所示为损失函数L的定义  
![](./img/contrastive_loss_02.webp)   
对于公式作如下解读：  
  + loss由两部分组成；
  + Lp表示对相似的图片(正样本)得到的特征值太远的惩罚（理论应该是越近越好）；
  + Ln表示对不相似的图片（负样本)得到的特征值太近的惩罚（理论应该至少大于m）;  
![](./img/contrastive_loss_03.webp)   
正样本(p,q)通过CNN得到的特征值x如果离得太远，会在loss优化的时候慢慢拉回来；  
负样本(p,n)通过CNN得到的特征值x如果小于阈值m，会在loss优化的时候慢慢拉大； 

## 3.2 siamese network
孪生网络顾名思义就是使用一样的网络结构，因为在我们的算法中，需要同时对需要正样本（p,q）或者是负样本(p,n)计算特征值，为了提高效率，设计了这样的结构~  
![](./img/contrastive_loss_04.webp)   
其实本质就是两个CNN网络共享参数θ，这个结构在上面那张图中也已经展示出来了。

# 4. 网络结构介绍
## 4.1 网络结构
在上面background的基础上，文章提出了4种网络结构：  
![](./img/contrastive_loss_05.webp)  

都不是很复杂，上面的看懂的话应该很快就能看懂这4个结构图。

A：将本来用于分类任务网络，将最后一层softmax改成特征提取层  
B：最基本的siamese CNN + contrastive loss结构  
C：在B的基础上增加了一个图像分类任务，用多任务更好的表示loss  
D：在C的基础上对提取到的特征多做了一次L2归一化操作  
文章最后给出的应用演示中用的是D结构，如下图所示为用D结构，并结合t-SNE算法做出来的可视化效果图:

![](./img/contrastive_loss_06.webp)  

## 4.2 重要处理细节
### 正负样本处理
我们直接拿到的标注数据一般只有正样本对(p,q),那么如何更好的利用这些正样本数据及生成负样本数据呢？  
个人觉得本文的处理方式还是挺有参考意义的，所以整理在这里：  
1. 生成负样本：对于每个图片p，都随机选取80张属于同一类别的图像+随机选取20张不属于这个类别的图像，这样每个图像p，都能生成100对负样本(p,n)；
2. 为了平衡正样本与负样本之比，正样本对\*5，这样正样本对：负样本对=5:100=1:20；
3. 对于每个图像，我们都有这个物体对应的框。为了数据增强，采用在原来的框外面额外选取像素大小为{0,8,16,32,48,64,80}的边框，这边边框其实就作为物品的背景，从而达到训练数据增强的效果；
4. 最后将数据分成train、val、test(图像p应该没有交叉)

### 其它小细节
1. margin(m)的初始化，文章探索了{1,√10, √100,√1000}, 最后用D结构的时候用了√2
2. 网络结构的初始化可以考虑用模型迁移
3. 距离度量使用的是cosine
4. 处理在测试阶段对region加了padding，在测试阶段也用了16的padding效果最好
